In [1]:
from datetime import date
from pandas import json_normalize
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df_work_raw = catalog.load('raw/openalex/work#parquet')

                    INFO     Loading data from raw/openalex/work#parquet (ParquetDataset)...    ]8;id=368608;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=281503;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

In [3]:
df_work_raw = df_work_raw.convert_dtypes()
df_work_raw = df_work_raw.loc[:,['id','locations']]

In [4]:
df_work_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         200 non-null    string
 1   locations  200 non-null    object
dtypes: object(1), string(1)
memory usage: 3.2+ KB


In [5]:
df_work_raw

,id,locations
0,https://openalex.org/W2089852843,"[{'id': 'doi:10.1016/s0378-4371(98)00437-3', '..."
1,https://openalex.org/W3037375387,"[{'id': 'doi:10.1016/j.envint.2020.105876', 'i..."
2,https://openalex.org/W2032143374,"[{'id': 'doi:10.1016/s0023-6438(03)00088-4', '..."
3,https://openalex.org/W2021071494,"[{'id': 'doi:10.1016/j.compedu.2005.11.017', '..."
4,https://openalex.org/W2154507013,"[{'id': 'doi:10.1126/science.1134796', 'is_acc..."
...,...,...
195,https://openalex.org/W2072634766,"[{'id': 'doi:10.4067/s0716-078x2000000100002',..."
196,https://openalex.org/W2592868344,"[{'id': 'doi:10.1007/s40626-016-0081-7', 'is_a..."
197,https://openalex.org/W2892467442,[{'id': 'doi:10.1016/j.neuroscience.2018.09.02...
198,https://openalex.org/W2015061845,"[{'id': 'doi:10.1016/j.foodcont.2009.07.012', ..."


In [6]:
df_work_authorship = df_work_raw.explode('locations').reset_index(drop=True)

In [7]:
def openalex_load_work_location(df_work_raw):

    df_work_raw.rename(columns={'id':'work_id'}, inplace=True)
    df_work_location = df_work_raw.explode('locations').reset_index(drop=True)

    df_work_location = pd.concat([df_work_location['work_id'], json_normalize(df_work_location['locations'])], axis=1)

    df_work_location.columns = df_work_location.columns.str.replace('.', '_')

    df_work_location = df_work_location[[
        'work_id','id', 
        'source_id', 'source_display_name', 'source_is_core', 'source_type',
        'source_host_organization', 'source_host_organization_name',
        'is_accepted', 'is_oa', 'is_published', 'landing_page_url',
        'license', 'license_id', 'pdf_url', 'version',
        # 'source_host_organization_lineage', 'source_host_organization_lineage_names', 'source_issn',
        'source_is_in_doaj', 'source_is_oa', 'source_issn_l'
    ]]

    df_work_location['load_datetime'] = date.today()

    return df_work_location


In [8]:
stage_work_location = openalex_load_work_location(df_work_raw)
stage_work_location

,work_id,id,source_id,source_display_name,source_is_core,source_type,source_host_organization,source_host_organization_name,is_accepted,is_oa,is_published,landing_page_url,license,license_id,pdf_url,version,source_is_in_doaj,source_is_oa,source_issn_l,load_datetime
0,https://openalex.org/W2089852843,doi:10.1016/s0378-4371(98)00437-3,https://openalex.org/S24688465,Physica A Statistical Mechanics and its Applic...,True,journal,https://openalex.org/P4310320990,Elsevier BV,True,False,True,https://doi.org/10.1016/s0378-4371(98)00437-3,None,None,None,publishedVersion,False,False,0378-4371,2025-11-23
1,https://openalex.org/W3037375387,doi:10.1016/j.envint.2020.105876,https://openalex.org/S143381477,Environment International,True,journal,https://openalex.org/P4310320990,Elsevier BV,True,True,True,https://doi.org/10.1016/j.envint.2020.105876,cc-by-nc-nd,https://openalex.org/licenses/cc-by-nc-nd,https://ars.els-cdn.com/content/image/1-s2.0-S...,publishedVersion,True,True,0160-4120,2025-11-23
2,https://openalex.org/W3037375387,pmid:32590284,https://openalex.org/S4306525036,PubMed,False,repository,https://openalex.org/I1299303238,National Institutes of Health,True,False,True,https://pubmed.ncbi.nlm.nih.gov/32590284,None,None,None,publishedVersion,False,False,None,2025-11-23
3,https://openalex.org/W3037375387,pmh:oai:ri.conicet.gov.ar:11336/142501,https://openalex.org/S4306401400,Conicet,False,repository,https://openalex.org/I4210104196,Centro Científico Tecnológico - Nordeste,False,True,False,http://hdl.handle.net/11336/142501,cc-by-nc-nd,https://openalex.org/licenses/cc-by-nc-nd,None,submittedVersion,False,False,None,2025-11-23
4,https://openalex.org/W3037375387,pmh:oai:doaj.org/article:9410c9207e9e450e8f655...,https://openalex.org/S4306401280,DOAJ (DOAJ: Directory of Open Access Journals),False,repository,None,None,False,True,False,https://doaj.org/article/9410c9207e9e450e8f655...,cc-by-sa,https://openalex.org/licenses/cc-by-sa,None,submittedVersion,False,False,None,2025-11-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
406,https://openalex.org/W2592868344,pmh:oai:sedici.unlp.edu.ar:10915/141886,https://openalex.org/S4306400803,El Servicio de Difusión de la Creación Intelec...,False,repository,https://openalex.org/I874386039,Universidad Nacional de La Plata,False,True,False,http://sedici.unlp.edu.ar/handle/10915/141886,cc-by,https://openalex.org/licenses/cc-by,None,submittedVersion,False,False,None,2025-11-23
407,https://openalex.org/W2892467442,doi:10.1016/j.neuroscience.2018.09.027,https://openalex.org/S30122339,Neuroscience,True,journal,https://openalex.org/P4310320990,Elsevier BV,True,True,True,https://doi.org/10.1016/j.neuroscience.2018.09...,None,None,https://www.sciencedirect.com/science/article/...,publishedVersion,False,False,0306-4522,2025-11-23
408,https://openalex.org/W2892467442,pmid:30268780,https://openalex.org/S4306525036,PubMed,False,repository,https://openalex.org/I1299303238,National Institutes of Health,True,False,True,https://pubmed.ncbi.nlm.nih.gov/30268780,None,None,None,publishedVersion,False,False,None,2025-11-23
409,https://openalex.org/W2015061845,doi:10.1016/j.foodcont.2009.07.012,https://openalex.org/S119525064,Food Control,True,journal,https://openalex.org/P4310320990,Elsevier BV,True,False,True,https://doi.org/10.1016/j.foodcont.2009.07.012,None,None,None,publishedVersion,False,False,0956-7135,2025-11-23
